In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

In [5]:
URL = "https://www.music-jobs.com/uk/jobsboard?page=2"
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
page = requests.get(URL, headers=agent)
soup = BeautifulSoup(page.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="width=device-width, initial-scale=1.0, user-scalable=yes" name="viewport"/>
  <title>
   UK Music Industry Magazine
  </title>
  <meta content="UK Music Industry Magazine" property="og:title"/>
  <meta content="" property="og:description"/>
  <meta content="https://www.music-jobs.com/uk" property="og:url"/>
  <meta content="https://s3-eu-west-1.amazonaws.com/mjmm-cdn-ireland/images/theme/skin/mmm-responsive/open-graph/OG-template_spurkle.jpg" property="og:image"/>
  <meta content="image/jpeg" property="og:image:type"/>
  <meta content="1200" property="og:image:width"/>
  <meta content="630" property="og:image:height"/>
  <meta content="Music Jobs" property="og:image:alt"/>
  <meta content="UK Music Industry Magazine" name="twitter:title"/>
  <meta content="@UKMusicJobs" name

In [6]:
def extract_job_titles_from_result(soup): 
    job_titles = []
    spans = soup.findAll("a", attrs={"class": "job-title"})
    for span in spans:
        job_titles.append(span.text.strip())
    return(job_titles)
job_titles = extract_job_titles_from_result(soup)

In [7]:
print(job_titles)
print(len(job_titles))

['Visiting Music Teacher of Piano', 'Event Officer (Fixed Term)', 'Research Fellow in Room Acoustic Modelling', 'Lead Music Leader', 'Programme Leader - Further Education', 'Quality Assurance Officer', 'Manual QA Tester', 'Music Tutors', 'Stage Manager: Live & Digital', 'Trainees', 'Salesforce Administrator', 'Strategic Buyer', 'Finance Trainee', 'Teacher of Oboe', 'Post Production Support Engineer', 'Senior Sound Designer - Gaming', 'Junior Producer', 'Producer', 'Product Manager, Core Fan Experience', 'B2B Marketing Operations Specialist']
20


In [8]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll("span", attrs={"class": "job-contract-location"})
    for span in spans:
        locations.append(span.text.strip())
    return(locations)
locations = extract_location_from_result(soup)

In [9]:
print(locations)
print(len(locations))

['West Sussex', 'Oxford', 'Guildford', 'Stockton on Tees', 'Manchester', 'Brighton', 'London', 'Middlesbrough', 'London', 'London/Remote', 'London', 'Salisbury', 'Salisbury', 'Godalming', 'London', 'Wimslow', 'London', 'Kings Cross, London', 'London', 'London']
20


In [13]:
uk_music_jobs = pd.DataFrame()

def extractor(range1, range2):
    columns=['jobTitle','location']

    final = pd.DataFrame()

    for page_number in range(range1,range2):
        uk_music_jobs = pd.DataFrame(columns=columns)
        
        agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}
        page = requests.get("https://www.music-jobs.com/uk/jobsboard?page=" + str(page_number), headers=agent)
        time.sleep(5)
        soup = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")

        job_titles = extract_job_titles_from_result(soup)
        locations = extract_location_from_result(soup)
        
        uk_music_jobs['jobTitle'] = list(job_titles)
        uk_music_jobs['location'] = list(locations)

        final = final.append(uk_music_jobs)
    
    return final
#https://stackoverflow.com/questions/44865673/access-denied-while-scraping

In [14]:
uk_mus_jobs = extractor(1, 24)

In [15]:
print(uk_mus_jobs.shape)

(444, 2)


In [16]:
uk_mus_jobs.head

<bound method NDFrame.head of                                              jobTitle  \
0                          Peripatetic Violin Teacher   
1   Audio Director [Unannounced Project] (365) (Au...   
2   Audio Director [Unannounced Project] (365) (Au...   
3                 Trainee Producer for micro-festival   
4                            Music Leader (Assistant)   
5                                 Music Teacher (KS2)   
6                                        Music Editor   
7                                   Lead Music Leader   
8                              Assistant Music Leader   
9                                   Lead Music Leader   
10                                           Producer   
11                               Development Director   
12   Marketing, Media and Digital Content Coordinator   
13  Emergent Commission Opportunity for Disabled a...   
14                    Senior Product Designer, Mobile   
15                                     Audio Director   
1

In [17]:
uk_mus_jobs['jobTitle'].value_counts()

Marketing Manager                                            5
Teacher of Music                                             5
Piano Teacher                                                4
Finance Assistant                                            4
Account Manager                                              3
Lead Music Leader                                            3
Music Teacher                                                3
Guitar Teacher                                               3
Producer                                                     3
Head of Music                                                2
Label Coordinator                                            2
Singing Tutor                                                2
Label Manager                                                2
Digital Marketing Manager                                    2
Social Media Manager                                         2
Marketing Project Manager                              

In [18]:
uk_mus_jobs['location'].value_counts()

London                                                                            192
Manchester                                                                         22
Remote                                                                             18
Berlin                                                                             10
High Wycombe                                                                        9
York                                                                                6
Middlesbrough                                                                       6
Leicester                                                                           6
Amsterdam                                                                           5
Bristol                                                                             5
Yorkshire                                                                           5
East Grinstead                                        

In [20]:
uk_mus_jobs.to_csv("uk_mus_jobs.csv", encoding="utf-8")


Source for all Scraping:

https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b